In [8]:
import pandas as pd

# Load the CSV file
file_path = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\IDS-DRR-Uttar-Pradesh-Risk-Score-Model\RiskScoreModel\data\risk_score_final_district.csv"
print("Loading CSV file from:", file_path)
df = pd.read_csv(file_path)
print("CSV file loaded successfully. Shape:", df.shape)

Loading CSV file from: D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\IDS-DRR-Uttar-Pradesh-Risk-Score-Model\RiskScoreModel\data\risk_score_final_district.csv
CSV file loaded successfully. Shape: (19890, 44)


C:\Users\saura\AppData\Local\Temp\ipykernel_30884\2943975513.py:6: DtypeWarning: Columns (8,9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [9]:
df.columns


Index(['object-id', 'district', 'timeperiod', 'total-tender-awarded-value',
       'max-rain', 'mean-rain', 'count', 'sum-rain', 'stname', 'dtname',
       'st-area-sh', 'st-length-', 'remarks', 'dist-lgd', 'state-lgd',
       'subdt-lgd', 'ac-no', 'st-length(shape)', 'sum-population', 'stcode11',
       'dtcode11', 'sub-district-code', 'sdtname', 'st-area(shape)',
       'urban-hhd', 'urban-electricity', 'urban-tele', 'urban-hhd-pipe',
       'urban-no-sanitation', 'rural-hhd', 'total-hhd', 'avg-electricity',
       'avg-tele', 'sd-piped-hhds-pct', 'sd-nosanitation-hhds-pct',
       'total-hhd.1', 'exposure', 'flood-hazard', 'government-response',
       'vulnerability', 'financial-year',
       'total-tender-awarded-value-fy-cumsum', 'topsis-score', 'risk-score'],
      dtype='object')

In [10]:
# Selecting only numeric columns along with 'object-id', 'timeperiod', and 'financial-year'
numeric_columns = df.select_dtypes(include=["number"]).columns
#list(numeric_columns).remove("shape-area")
#numeric_columns = numeric_columns.drop("shape-area")
df_numeric = df[["object-id", "timeperiod", "financial-year"] + list(numeric_columns)]

# Identifying columns that are not included
excluded_columns = [col for col in df.columns if col not in df_numeric.columns]
print("Excluded columns:", excluded_columns)

df_numeric.head()

Excluded columns: ['district', 'stname', 'dtname', 'sdtname']


,object-id,timeperiod,financial-year,total-tender-awarded-value,max-rain,mean-rain,count,sum-rain,st-area-sh,st-length-,...,sd-piped-hhds-pct,sd-nosanitation-hhds-pct,total-hhd.1,exposure,flood-hazard,government-response,vulnerability,total-tender-awarded-value-fy-cumsum,topsis-score,risk-score
0,09-195-00990,2021_04,2021-2022,0.0,0.0,0.0,775.0,0.18,0.0,0.0,...,12.29,9.93,147093.0,2,1.0,5,4,0.0,0.928971,5
1,09-192-00973,2021_04,2021-2022,0.0,0.0,0.0,474.0,0.10,0.0,0.0,...,11.54,8.96,114526.0,2,1.0,5,4,0.0,0.928971,5
2,09-183-00931,2021_04,2021-2022,0.0,0.0,0.0,658.0,0.21,0.0,0.0,...,10.57,12.32,156431.0,2,1.0,5,4,0.0,0.928971,5
3,09-185-00938,2021_04,2021-2022,0.0,0.0,0.0,1143.0,0.35,0.0,0.0,...,7.94,7.37,211401.0,2,1.0,5,3,0.0,0.919837,5
4,09-177-00905,2021_04,2021-2022,0.0,0.0,0.0,442.0,0.11,0.0,0.0,...,15.70,9.91,110504.0,2,1.0,5,3,0.0,0.919837,5


In [11]:

df_melted = df_numeric.melt(id_vars=["object-id", "timeperiod", "financial-year"], var_name="factor", value_name="score")
print("Data melted. Shape:", df_melted.shape)

df_melted.head()

Data melted. Shape: (735930, 5)


,object-id,timeperiod,financial-year,factor,score
0,09-195-00990,2021_04,2021-2022,total-tender-awarded-value,0.0
1,09-192-00973,2021_04,2021-2022,total-tender-awarded-value,0.0
2,09-183-00931,2021_04,2021-2022,total-tender-awarded-value,0.0
3,09-185-00938,2021_04,2021-2022,total-tender-awarded-value,0.0
4,09-177-00905,2021_04,2021-2022,total-tender-awarded-value,0.0


In [12]:

df_transposed = df_melted.pivot(index=["factor", "timeperiod", "financial-year"], columns="object-id", values="score").reset_index()
print("Data pivoted. Shape:", df_transposed.shape)
df_transposed.head()
# Save or display the transformed data
output_file = r"D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\IDS-DRR-Uttar-Pradesh-Risk-Score-Model\RiskScoreModel\data\Transformed_UP_Data.csv"
df_transposed.to_csv(output_file, index=False)
print("Transformed data saved to:", output_file)

Data pivoted. Shape: (1887, 393)
Transformed data saved to: D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\IDS-DRR-Uttar-Pradesh-Risk-Score-Model\RiskScoreModel\data\Transformed_UP_Data.csv


In [7]:
# Verify with the source data. Given factor, district, timeperiod and object-id, the score should match

factor = "sdrf-sanctions-awarded-value"
district = '02-031-00202'
timeperiod = '2024_07'
df_transposed.head()
print("modified")
print(df_transposed[(df_transposed["factor"] == factor) & (df_transposed["timeperiod"] == timeperiod) ][[district, "timeperiod", "financial-year", "factor"]])
print("original")
df[(df["object-id"] == district) & (df["timeperiod"] == timeperiod)][['timeperiod', 'financial-year', factor]]



modified


KeyError: "['02-031-00202'] not in index"

In [8]:
dff = pd.read_csv("Transformed_Assam_Data.csv")
conditions = []
operator_map = {
            '==': lambda col, val: col == val,
            '!=': lambda col, val: col != val,
            '>': lambda col, val: col > val,
            '<': lambda col, val: col < val,
            '>=': lambda col, val: col >= val,
            '<=': lambda col, val: col <= val,
            'in': lambda col, val: col.isin(val),
            'not in': lambda col, val: ~col.isin(val)
        }
conditions.append(operator_map["=="](dff["factor"], "sdrf-sanctions-awarded-value"))

dff = dff[pd.concat(conditions, axis=1).all(axis=1)]
dff

,factor,timeperiod,financial-year,18-300,18-300-00101,18-300-00102,18-300-00103,18-300-00104,18-300-00105,18-300-00106,...,18-799,18-799-00124,18-799-00125,18-799-00265,18-816,18-816-00235,18-816-00236,18-816-00258,18-816-00259,18-816-00261
2835,sdrf-sanctions-awarded-value,2021_04,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2836,sdrf-sanctions-awarded-value,2021_05,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2837,sdrf-sanctions-awarded-value,2021_06,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2838,sdrf-sanctions-awarded-value,2021_07,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2839,sdrf-sanctions-awarded-value,2021_08,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2840,sdrf-sanctions-awarded-value,2021_09,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2841,sdrf-sanctions-awarded-value,2021_10,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2842,sdrf-sanctions-awarded-value,2021_11,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2843,sdrf-sanctions-awarded-value,2021_12,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2844,sdrf-sanctions-awarded-value,2022_01,2021-2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
